## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Laukaa,RU,58.7372,29.8453,45.37,95,100,5.93,overcast clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,24.80,80,100,10.36,overcast clouds
2,2,Saskylakh,RU,71.9167,114.0833,32.16,93,100,10.96,overcast clouds
3,3,Kapaa,US,22.0752,-159.3190,82.38,70,0,14.97,clear sky
4,4,Mersing,MY,2.4312,103.8405,89.51,79,40,5.75,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.0752,-159.3190,82.38,70,0,14.97,clear sky
4,4,Mersing,MY,2.4312,103.8405,89.51,79,40,5.75,scattered clouds
6,6,Bambous Virieux,MU,-20.3428,57.7575,73.65,83,40,8.05,scattered clouds
8,8,Beloha,MG,-25.1667,45.0500,63.21,95,4,5.61,clear sky
9,9,Saint Anthony,US,45.0205,-93.2180,66.13,42,0,13.80,clear sky
13,13,Rikitea,PF,-23.1203,-134.9692,76.15,75,100,20.96,overcast clouds
14,14,Shache,CN,38.4167,77.2406,70.93,37,100,12.21,overcast clouds
16,16,Hays,US,30.0502,-98.0336,82.74,63,71,8.77,broken clouds
17,17,Martapura,ID,-3.4167,114.8500,84.29,79,20,4.61,few clouds
19,19,Hovd,MN,48.0056,91.6419,62.49,12,0,4.34,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,82.38,clear sky,22.0752,-159.3190,
4,Mersing,MY,89.51,scattered clouds,2.4312,103.8405,
6,Bambous Virieux,MU,73.65,scattered clouds,-20.3428,57.7575,
8,Beloha,MG,63.21,clear sky,-25.1667,45.0500,
9,Saint Anthony,US,66.13,clear sky,45.0205,-93.2180,
13,Rikitea,PF,76.15,overcast clouds,-23.1203,-134.9692,
14,Shache,CN,70.93,overcast clouds,38.4167,77.2406,
16,Hays,US,82.74,broken clouds,30.0502,-98.0336,
17,Martapura,ID,84.29,few clouds,-3.4167,114.8500,
19,Hovd,MN,62.49,clear sky,48.0056,91.6419,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():    

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df #.notnull().sum()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,82.38,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Mersing,MY,89.51,scattered clouds,2.4312,103.8405,Timotel Hotel
6,Bambous Virieux,MU,73.65,scattered clouds,-20.3428,57.7575,Casa Tia Villa
8,Beloha,MG,63.21,clear sky,-25.1667,45.0500,
9,Saint Anthony,US,66.13,clear sky,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
...,...,...,...,...,...,...,...
696,San Cristobal,VE,73.58,overcast clouds,7.7669,-72.2250,Pirineos
697,Kisangani,CD,69.89,light rain,0.5167,25.2000,Grand Ramela Hotel
699,Elizabeth City,US,69.82,light rain,36.2946,-76.2511,Hampton Inn Elizabeth City
701,Soyo,AO,76.77,scattered clouds,-6.1349,12.3689,Hotel Mpampa


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df[hotel_df["Hotel Name"]!= '']
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,82.38,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Mersing,MY,89.51,scattered clouds,2.4312,103.8405,Timotel Hotel
6,Bambous Virieux,MU,73.65,scattered clouds,-20.3428,57.7575,Casa Tia Villa
9,Saint Anthony,US,66.13,clear sky,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
13,Rikitea,PF,76.15,overcast clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
696,San Cristobal,VE,73.58,overcast clouds,7.7669,-72.2250,Pirineos
697,Kisangani,CD,69.89,light rain,0.5167,25.2000,Grand Ramela Hotel
699,Elizabeth City,US,69.82,light rain,36.2946,-76.2511,Hampton Inn Elizabeth City
701,Soyo,AO,76.77,scattered clouds,-6.1349,12.3689,Hotel Mpampa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))